In [45]:
import os
import nltk
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import random
from nltk.corpus import stopwords
import operator


stopwords = nltk.corpus.stopwords.words('english')
new_stopwords = ['/','(',')','{','}','@','|',';','\n','#','+','_','.',',','``',"''",]
stopwords.extend(new_stopwords)
lemmatizer = nltk.stem.WordNetLemmatizer()

path = os.getcwd() + "\\bbc"
list_catagory_files = os.listdir(path)
print(list_catagory_files)
num_catagory = len(list_catagory_files)

def text_label_list(cata_path):
    label = 0
    all_text_list = []
    all_label_list = []
    catagory_file_num = []
    
    for catagory in list_catagory_files:
        catagory_file = path + "\\" + catagory
        print(catagory_file)
        if catagory_file.endswith('.TXT'):
            continue
        else:
        
            file_num = len(os.listdir(catagory_file))
            catagory_file_num.append(file_num)
            print(file_num)
            print(catagory)
            for i in range(1,file_num + 1):
                all_label_list.append(label)
                j = "%03d" % i
                with open("\\".join([catagory_file, str(j) + ".txt"])) as fp:
                    article = fp.readlines()
                    all_text_list.append(article)
        label += 1
    return all_text_list, all_label_list, catagory_file_num

all_text_list_, all_label_list_, catagory_file_num = text_label_list(list_catagory_files)      
        
# get full dataset in order to split our dataset into training and test
dataset_full=[] 
for text, label in zip(all_text_list_, all_label_list_):
    dataset_full.append((text,label))
        

def get_train_test_split(dataset_full,ratio):
    pre_train_set=[]
    pre_test_set=[]
    
    size_dataset_full = len(dataset_full)
    test_size = int(round(size_dataset_full * ratio))
    list_test_indices=random.sample(range(size_dataset_full), test_size)
    
    for i,text in enumerate(dataset_full):
        if i in list_test_indices:
            pre_test_set.append(text)
        else:pre_train_set.append(text)
        
    return pre_train_set,pre_test_set
# get training set, test set
train_set, test_set = get_train_test_split(dataset_full,0.2)
print ("Size training set: "+str(len(train_set)))
print ("Size test set: "+str(len(test_set)))


# get training set, test set, development set
new_test_set, new_dev_set = get_train_test_split(test_set,0.5)

new_train_set=train_set
random.shuffle(new_train_set)
random.shuffle(new_dev_set)
random.shuffle(new_test_set)
print ("Size training set: "+str(len(train_set)))
print ("Size test set: "+str(len(new_test_set)))
print ("Size development set: "+str(len(new_dev_set)))



def get_list_tokens(text_list):
    paragraph = ",".join(text_list)
    sentence_split=nltk.tokenize.sent_tokenize(paragraph)
    
    list_tokens=[]
    for sentence in sentence_split:
        list_tokens_sentence=nltk.tokenize.word_tokenize(sentence)
        for token in list_tokens_sentence:
            list_tokens.append(lemmatizer.lemmatize(token).lower())
    return list_tokens

def get_vector_text(list_vocab,text_list):
    vector_text=np.zeros(len(list_vocab))
    paragraph = ",".join(text_list)
    list_tokens_string=get_list_tokens(paragraph)
    for i, word in enumerate(list_vocab):
        if word in list_tokens_string:
            vector_text[i]=list_tokens_string.count(word)
    return vector_text


def get_vocabulary(training_set, num_features): # Function to retrieve vocabulary
    dict_word_frequency={}
    for instance in training_set:
        sentence_tokens=get_list_tokens(instance[0])
        for word in sentence_tokens:
            if word in stopwords: continue
            if word not in dict_word_frequency: dict_word_frequency[word]=1
            else: dict_word_frequency[word]+=1
    sorted_list = sorted(dict_word_frequency.items(), key=operator.itemgetter(1), reverse=True)[:num_features]
    vocabulary=[]
    for word,frequency in sorted_list:
        vocabulary.append(word)
    return vocabulary   #word frequency元组







def train_svm_classifier(training_set, vocabulary): # Function for training our svm classifier
    X_train=[]
    Y_train=[]
    for instance in training_set:
        vector_instance=get_vector_text(vocabulary,instance[0])
        X_train.append(vector_instance)
        Y_train.append(instance[1])
  # Finally, we train the SVM classifier 
    svm_clf=sklearn.svm.SVC(kernel="linear",gamma='auto')
    svm_clf.fit(np.asarray(X_train),np.asarray(Y_train))
    return svm_clf

vocabulary=get_vocabulary(new_train_set, 1000)
svm_clf=train_svm_classifier(new_train_set, vocabulary)

['business', 'entertainment', 'politics', 'README.TXT', 'sport', 'tech']
C:\Users\c2008016\Desktop\Programs\coursework\bbc\business
510
business
C:\Users\c2008016\Desktop\Programs\coursework\bbc\entertainment
386
entertainment
C:\Users\c2008016\Desktop\Programs\coursework\bbc\politics
417
politics
C:\Users\c2008016\Desktop\Programs\coursework\bbc\README.TXT
C:\Users\c2008016\Desktop\Programs\coursework\bbc\sport
511
sport
C:\Users\c2008016\Desktop\Programs\coursework\bbc\tech
401
tech
Size training set: 1780
Size test set: 445
Size training set: 1780
Size test set: 223
Size development set: 222


In [46]:
X_test=[]
Y_test=[]
for instance in new_test_set:   #注意是test_set,上面的描述也错了
    vector_instance=get_vector_text(vocabulary,instance[0])
    X_test.append(vector_instance)
    Y_test.append(instance[1])
X_test=np.asarray(X_test)
Y_test_gold=np.asarray(Y_test)

from sklearn.metrics import classification_report
Y_text_predictions=svm_clf.predict(X_test)  #重要
print(Y_text_predictions)
print(classification_report(Y_test_gold, Y_text_predictions))

[3 2 0 0 0 2 3 1 3 1 3 3 3 4 4 3 0 3 0 4 4 0 4 4 3 1 1 2 2 1 3 0 2 4 3 3 3
 3 0 2 3 2 3 3 2 3 0 0 3 3 3 2 3 4 4 3 4 4 2 3 4 1 3 3 1 4 3 3 3 1 4 3 3 3
 3 4 0 2 3 4 3 1 2 0 4 1 4 4 0 4 2 0 3 3 3 0 1 0 4 4 0 4 3 1 3 3 0 3 3 3 4
 4 4 0 3 0 3 3 0 1 0 3 3 3 2 3 4 3 0 0 3 0 3 2 3 3 0 3 0 3 3 2 0 3 1 0 0 1
 4 3 3 2 3 4 3 0 0 3 1 3 2 2 3 3 3 3 4 3 4 0 3 0 4 0 0 0 1 0 3 3 3 2 0 3 0
 3 1 2 3 4 3 3 0 3 1 2 4 1 0 4 4 3 0 4 1 4 3 3 3 3 2 3 4 0 0 0 3 0 0 0 0 3
 4]
              precision    recall  f1-score   support

           0       0.78      0.68      0.73        57
           1       0.48      0.26      0.33        39
           2       0.57      0.33      0.42        39
           3       0.51      0.80      0.62        56
           4       0.53      0.66      0.58        32

    accuracy                           0.57       223
   macro avg       0.57      0.55      0.54       223
weighted avg       0.58      0.57      0.56       223



In [47]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score

precision=precision_score(Y_test_gold, Y_text_predictions, average='macro')
recall=recall_score(Y_test_gold, Y_text_predictions, average='macro')
f1=f1_score(Y_test_gold, Y_text_predictions, average='macro')
accuracy=accuracy_score(Y_test_gold, Y_text_predictions)

print ("Precision: "+str(round(precision,3)))
print ("Recall: "+str(round(recall,3)))
print ("F1-Score: "+str(round(f1,3)))
print ("Accuracy: "+str(round(accuracy,3)))

Precision: 0.57
Recall: 0.547
F1-Score: 0.537
Accuracy: 0.574
